### Dependencies


In [1]:
# Data collection, cleaning, storage
import pandas as pd
import numpy as np
np.bool = np.bool_ # https://stackoverflow.com/questions/74893742/how-to-solve-attributeerror-module-numpy-has-no-attribute-bool
import psycopg2
# from dotenv import load_dotenv
# import os
# import yfinance as yf
import re
import os


import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import random
from gluonts.dataset.common import ListDataset
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

from sklearn.preprocessing import RobustScaler

#  App dependencies
#  Reload modules to avoid functions not found
import importlib

import functions.database
importlib.reload(functions.database)

from functions.database import (
    get_column_name_and_datatype_dictionary, 
    prepare_sql_queries_and_values,
    insert_data_into_sql_data_base,
    retrieve_data_from_sql,
    add_metadata_columns
)

import functions.machinelearning
importlib.reload(functions.machinelearning)

from functions.machinelearning import (
    prep_data_for_deep_ar_model,
    create_model_and_train,
    generate_forecasts,
    inverse_transform_forecasts,
    get_forecast_data_frames,

)

import functions.ml_evaluation
importlib.reload(functions.ml_evaluation)

from functions.ml_evaluation import (
    get_evaluation_metrics, 
    get_combined_rmse,
    get_experiment_number,
    get_hyperparameters,
)

import functions.applicant_assessment_results
importlib.reload(functions.applicant_assessment_results)

from functions.applicant_assessment_results import (
    assess_affordability,
    get_overall_assessment,
)

import warnings
warnings.filterwarnings('ignore')

### Get data from SQL


<!-- Get applicant data from DB -->




In [2]:
# import sql ingestion script 
# Data to read from csv so we dont have random data each time
# data = pd.read_csv("sample_account_transactions.csv")
# insert_applicant_fin_history(data, "fin_history")
# Note update data to have unique id for each applicant and no update on conflict with id and date

In [3]:
data = retrieve_data_from_sql("fin_history")
applicant_id = '123456799'
required_amount = 14000  # Example threshold amount



Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


In [4]:
# data.head()

,date,balance,transaction,day_of_month,day_of_week,is_weekend,rolling_7d_mean,rolling_7d_std,is_salary_day,is_rent_day,is_major_expense,trend_7d
0,2024-01-01,5000.000000,0.000000,1,0,False,4884.668728,113.68683,False,False,False,0.0
1,2024-01-02,4990.614638,-9.385362,2,1,False,4884.668728,113.68683,False,False,False,0.0
2,2024-01-03,4930.412210,-60.202429,3,2,False,4884.668728,113.68683,False,False,False,0.0
3,2024-01-04,4904.077296,-26.334914,4,3,False,4884.668728,113.68683,False,False,False,0.0
4,2024-01-05,4885.818445,-18.258851,5,4,False,4884.668728,113.68683,False,False,False,0.0


### Feature Engineering


### Prepare training data


In [5]:
# Step 2: Prepare data for DeepAR - 
# - Split data 
# - Prep dynamic features
# - normalise where neccessary, 
# - scale balance with appropriate scaler 
# - convert to expected GluonTS format

# Split data: use first 7 months for training, last month for validation
split_idx = len(data) - 30  # Last 30 days as validation
train_data = data.iloc[:split_idx]

training_data, scaler = prep_data_for_deep_ar_model(train_data)


### DeepAR Model building and training


In [6]:

forecasting_model_for_validation = create_model_and_train(training_data)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type        | Params | Mode  | In sizes                                                           | Out sizes   
----------------------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 14.0 K | train | [[1, 1], [1, 1], [1, 1182, 12], [1, 1182], [1, 1182], [1, 30, 12]] | [1, 100, 30]
----------------------------------------------------------------------------------------------------------------------------------
14.0 K    Trainable params
0         Non-trainable params
14.0 K    Total params
0.056     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 12: 'train_loss' reached 0.74981 (best 0.74981), saving model to '/Users/admin/PycharmProjects/unboundafford/lightning_logs/version_107/checkpoints/epoch=0-step=12.ckpt' as top 1
Epoch 1, global step 24: 'train_loss' reached 0.55392 (best 0.55392), saving model to '/Users/admin/PycharmProjects/unboundafford/lightning_logs/version_107/checkpoints/epoch=1-step=24.ckpt' as top 1
Epoch 2, global step 36: 'train_loss' reached 0.29908 (best 0.29908), saving model to '/Users/admin/PycharmProjects/unboundafford/lightning_logs/version_107/checkpoints/epoch=2-step=36.ckpt' as top 1
Epoch 3, global step 48: 'train_loss' reached -0.11578 (best -0.11578), saving model to '/Users/admin/PycharmProjects/unboundafford/lightning_logs/version_107/checkpoints/epoch=3-step=48.ckpt' as top 1
Epoch 4, global step 60: 'train_loss' reached -0.55514 (best -0.55514), saving model to '/Users/admin/PycharmProjects/unboundafford/lightning_logs/version_107/checkpoints/epoch=4-step=60.ckpt' as to

### Generate forecasts for validation


In [7]:
# Step 4.1: Generate forecasts



validation_forecasts, validation_tss = generate_forecasts(forecasting_model_for_validation, training_data)



In [8]:
# #  Test - not needed for final script in API 
# # We need to ensure the scaler is properly fitted
# # Let's refit it on the original training data
# from sklearn.preprocessing import RobustScaler

# # Get original unscaled values

# # This assumes you have the original data before scaling
# original_values = train_data['balance'].values.reshape(-1, 1)

# # Initialize and fit a new scaler
# scaler = RobustScaler()
# scaler.fit(original_values)

# # Transform a sample to verify the scaler works as expected
# sample_scaled = scaler.transform(original_values[:5])
# sample_restored = scaler.inverse_transform(sample_scaled)
# print("\nScaling verification:")
# print("Original:", original_values[:5].flatten())
# print("\n")
# print("Scaled:", sample_scaled.flatten())
# print("Restored:", sample_restored.flatten())

In [9]:
# Now transform the forecasts
# Step 4.2: Generate forecasts - Transform forecasts back to original scale

transformed_validation_forecast_values = inverse_transform_forecasts(validation_forecasts[0], scaler)

# # Check the transformed values
# print("\nTransformed forecast values:")
# print("p50 (first 5):", transformed_validation_forecast_values['p50'][:5])

In [10]:
(forecast_7days_validation_set, 
 forecast_14days_validation_set, 
 forecast_30days_validation_set
 ) = get_forecast_data_frames(transformed_validation_forecast_values, data)

In [11]:
# forecast_30days_validation_set.head()

,p1,p5,p10,p20,p30,p40,p50,p60,p70,p80,p90,p92,p95,p99,actual,date
0,11593.084961,12632.908203,13071.518555,13467.448242,13685.195312,13843.489258,13963.100586,14129.889648,14265.431641,14452.591797,14743.108398,14875.585938,15135.615234,15965.931641,15808.911708,2024-07-29
1,12530.459961,13456.638672,13735.852539,13972.800781,14157.493164,14294.830078,14449.661133,14574.613281,14714.414062,14912.550781,15142.994141,15237.324219,15517.115234,16582.437500,15807.127487,2024-07-30
2,11768.564453,12878.362305,13192.689453,13463.016602,13635.769531,13761.422852,13893.563477,14013.916992,14122.147461,14284.939453,14572.241211,14649.464844,14839.455078,15613.298828,15806.066185,2024-07-31
3,12228.866211,12761.113281,13031.379883,13273.333984,13431.655273,13532.746094,13640.089844,13737.688477,13860.089844,13989.825195,14243.558594,14301.850586,14494.091797,15381.904297,14537.994717,2024-08-01
4,11537.552734,12704.394531,12987.938477,13228.427734,13381.309570,13513.343750,13624.518555,13738.980469,13863.695312,14013.234375,14250.558594,14318.325195,14512.143555,15507.847656,14512.095506,2024-08-02


# Model validation & evaluation 


In [12]:
# selected_metrics = get_evaluation_metrics(validation_tss, validation_forecasts, scaler)

In [13]:
# selected_metrics

In [14]:
# Calculate RMSE

combined_rmse_df = get_combined_rmse(forecast_7days_validation_set, forecast_14days_validation_set, forecast_30days_validation_set)


# combined_rmse_df


,quartiles,seven_day_forecast,fourteen_day_forecast,thirty_day_forecast
0,p1,3156.702064,2742.225599,2791.761490
1,p5,2264.530481,1911.612032,1823.584030
2,p10,1965.946367,1635.869394,1511.423338
3,p20,1694.584509,1388.506706,1234.287854
4,p30,1530.023573,1235.570015,1072.589228
5,p40,1405.266940,1119.568849,958.366601
6,p50,1290.694493,1012.927319,861.687264
7,p60,1173.055925,905.785720,781.709637
8,p70,1060.574770,801.890283,719.953830
9,p80,912.351439,669.405648,676.198009


###  save model/ experiment results

In [15]:


# Define the path to the lightning_logs directory
logs_dir = "lightning_logs"



experiment_no = get_experiment_number(logs_dir)
# experiment_no

Highest experiment number found: 107


107

In [16]:
#  extract hyperparameters




hyperparameters_path = f'lightning_logs/version_{experiment_no}/hparams.yaml'
experiment_id = f'exp_{experiment_no}'
hyperparameters_df = get_hyperparameters(hyperparameters_path, experiment_id)

# print(df)
# print(df)
# hyperparameters_df.head(30)


,Category,Metric,Value,ExperimentID
0,Hyperparameter,lr,0.001,exp_107
1,Hyperparameter,model_kwargs.cardinality.context_length,90,exp_107
2,Hyperparameter,model_kwargs.cardinality.default_scale,None,exp_107
3,Hyperparameter,state.__init_args__,*id001,exp_107
4,Hyperparameter,state.beta,0.0,exp_107
5,Hyperparameter,dropout_rate,0.1,exp_107
6,Hyperparameter,embedding_dimension,None,exp_107
7,Hyperparameter,freq,D,exp_107
8,Hyperparameter,hidden_size,40,exp_107
9,Hyperparameter,lags_seq,None,exp_107


In [17]:
# Step 6: Extract key metrics for assessment using transformed values


# Final balance predictions (use the last value of each transformed forecast array)
final_median = transformed_validation_forecast_values['p50'][-1]
final_p10 = transformed_validation_forecast_values['p10'][-1]
final_p90 = transformed_validation_forecast_values['p90'][-1]


# Extract actual final balance if available
if len(data) > len(train_data):
    actual_final = data['balance'].iloc[-1]
    error = actual_final - final_p90
    
    # Check if actual falls within the prediction interval
    within_interval = final_p10 <= actual_final <= final_p90
  

# Example assessment for a required amount

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

affordability_assessment = assess_affordability(required_amount, final_p10, final_p90)



In [18]:
# Overall assessment 

experiment_id = f'exp_{experiment_no}'

overall_validation_forecast_assessment_df = get_overall_assessment(
    experiment_id,
    required_amount, 
    affordability_assessment, 
    train_data, 
    final_p10, 
    final_median, 
    final_p90, 
    actual_final,
    error, 
    within_interval,
    )

# overall_validation_forecast_assessment_df
# the one for the test set/future forecst will not have comparative metrics
   

,Category,Metric,Value,ExperimentID
0,Affordability,Required amount,£14000.00,exp_107
1,Affordability,Assessment,Moderate confidence,exp_107
2,Affordability,Probability of meeting threshold,50-90%,exp_107
3,Affordability,Recommendation,Approve with monitoring,exp_107
4,Affordability,Buffer amount (median forecast),£1949.74,exp_107
5,Forecast,Current balance,£15828.42,exp_107
6,Forecast,Forecast for 30 days later (median),£14715.68,exp_107
7,Forecast,Conservative forecast (10th percentile),£13406.41,exp_107
8,Forecast,Optimistic forecast (90th percentile),£15949.74,exp_107
9,Forecast,Forecast range width,£2543.33,exp_107


In [19]:
#  Combine overall results and hyperparametres into a data frame

hyperparameters_and_overall_validation_assessment_df = pd.concat([hyperparameters_df, overall_validation_forecast_assessment_df], ignore_index=True)

# Save to csv
hyperparameters_and_overall_validation_assessment_df.to_csv(f'result_exp_{experiment_no}.csv', index=False) 

In [20]:

# hyperparameters_and_overall_validation_assessment_df.head(50)

In [21]:
# insert in database 
# ................MODIFY TO HAVE UNIQUE ID FOR EACH APPLICANT................
#  1. Forecasts x 30 days 
#  2. feature engineered data (actual balance including 7days rolling average)
#  3. Converted to gbp
#  3. Hyperparameters & overall validation assessment

# Database tables 

dbname='cwb-database',

#  delete/DROP later
'cwb_results' # old table for holding results

##### Active tables
# Financial history 
fin_history_table_name = 'fin_history'
fin_history_enhanced_table_name = 'fin_history_enhanced' # Feature engineered data, applicant id, date, time etc

# Applicant Assessment and Model Evaluation 
cwb_combined_rmse_table_name = 'cwb_combined_rmse' # Combined results for validation and future set
cwb_validation_assessment_table_name = 'cwb_validation_assessment' # Assessment and Hyperparameters for validation set
# cwb_future_assessment_table_name = 'cwb_future_assessment' # Assessment and Hyperparameters for future set

# Forecasts
cwb_validation_forecasts_table_name = 'cwb_validation_forecasts' # 30 days forecast, date, actual balance
# gbp_cwb_validation_forecasts_table_name = 'gbp_cwb_validation_forecasts' # 30 days forecast, date, actual balance
# cwb_future_forecasts_table_name = 'cwb_future_forecasts'  # 30 days forecast, date
# gbp_cwb_future_forecasts_table_name = 'gbp_cwb_future_forecasts'  # 30 days forecast, date






In [22]:
# Insert into database

# Add meta data to Financial history enhanced
data_df = add_metadata_columns(data, applicant_id = applicant_id)

# Add metadata columns to combined RMSE
combined_rmse_df = add_metadata_columns(combined_rmse_df, applicant_id = applicant_id)

# Add meta data to relevant dataframes 
hyperparameters_and_overall_validation_assessment_df = add_metadata_columns(hyperparameters_and_overall_validation_assessment_df, applicant_id = applicant_id)

# Add metadata columns to validation forecasts
forecast_30days_validation_set_df = add_metadata_columns(forecast_30days_validation_set, applicant_id = applicant_id)


# 1. Financial history enhanced
cwb_fin_history_enhanced_dict = get_column_name_and_datatype_dictionary(data_df)
cwb_fin_history_enhanced_sql_queries_and_values = prepare_sql_queries_and_values(cwb_fin_history_enhanced_dict, fin_history_enhanced_table_name, data_df)
insert_data_into_sql_data_base(*cwb_fin_history_enhanced_sql_queries_and_values)

# 2. Combined RMSE
cwb_rmse_dict = get_column_name_and_datatype_dictionary(combined_rmse_df)
cwb_rsme_sql_queries_and_values = prepare_sql_queries_and_values(cwb_rmse_dict, cwb_combined_rmse_table_name, combined_rmse_df)
insert_data_into_sql_data_base(*cwb_rsme_sql_queries_and_values)

# 3. Validation Assessment Results (and Hyperparameters)
cwb_validation_results_dict = get_column_name_and_datatype_dictionary(hyperparameters_and_overall_validation_assessment_df)
cwb_validation_results_sql_queries_and_values = prepare_sql_queries_and_values(cwb_validation_results_dict, cwb_validation_assessment_table_name, hyperparameters_and_overall_validation_assessment_df)
insert_data_into_sql_data_base(*cwb_validation_results_sql_queries_and_values)

# 4. Validation Forecasts
cwb_validation_forecasts_dict = get_column_name_and_datatype_dictionary(forecast_30days_validation_set_df)
cwb_validation_forecasts_sql_queries_and_values = prepare_sql_queries_and_values(cwb_validation_forecasts_dict, cwb_validation_forecasts_table_name, forecast_30days_validation_set_df)
insert_data_into_sql_data_base(*cwb_validation_forecasts_sql_queries_and_values)




Connected to database...'cwb-database'
Inserting 240 rows in a batch...into table 'fin_history_enhanced'
Your data has been inserted successfully into the SQL database...
Your connection is closed


Connected to database...'cwb-database'
Inserting 14 rows in a batch...into table 'cwb_combined_rmse'
Your data has been inserted successfully into the SQL database...
Your connection is closed


Connected to database...'cwb-database'
Inserting 33 rows in a batch...into table 'cwb_validation_assessment'
Your data has been inserted successfully into the SQL database...
Your connection is closed


Connected to database...'cwb-database'
Inserting 30 rows in a batch...into table 'cwb_validation_forecasts'
Your data has been inserted successfully into the SQL database...
Your connection is closed



In [23]:
retrieve_data_from_sql(cwb_validation_assessment_table_name)


Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


,category,metric,value,experimentid,applicant_id,date_added,transaction_time,sn
0,Hyperparameter,lr,0.001,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,1
1,Hyperparameter,model_kwargs.cardinality.context_length,90,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,2
2,Hyperparameter,model_kwargs.cardinality.default_scale,None,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,3
3,Hyperparameter,state.__init_args__,*id001,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,4
4,Hyperparameter,state.beta,0.0,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,5
5,Hyperparameter,dropout_rate,0.1,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,6
6,Hyperparameter,embedding_dimension,None,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,7
7,Hyperparameter,freq,D,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,8
8,Hyperparameter,hidden_size,40,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,9
9,Hyperparameter,lags_seq,None,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,10


In [27]:
retrieve_data_from_sql(cwb_validation_assessment_table_name)


Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


,category,metric,value,experimentid,applicant_id,date_added,transaction_time,sn
0,Hyperparameter,lr,0.001,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,1
1,Hyperparameter,model_kwargs.cardinality.context_length,90,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,2
2,Hyperparameter,model_kwargs.cardinality.default_scale,None,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,3
3,Hyperparameter,state.__init_args__,*id001,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,4
4,Hyperparameter,state.beta,0.0,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,5
5,Hyperparameter,dropout_rate,0.1,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,6
6,Hyperparameter,embedding_dimension,None,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,7
7,Hyperparameter,freq,D,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,8
8,Hyperparameter,hidden_size,40,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,9
9,Hyperparameter,lags_seq,None,exp_107,123456799,2025-03-28,2025-03-28 21:34:23.683536,10


In [28]:
retrieve_data_from_sql(cwb_combined_rmse_table_name)


Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


,quartiles,seven_day_forecast,fourteen_day_forecast,thirty_day_forecast,applicant_id,date_added,transaction_time,sn
0,p60,1173.055925,905.785720,781.709637,123456799,2025-03-28,2025-03-28 21:34:23.679749,8
1,p70,1060.574770,801.890283,719.953830,123456799,2025-03-28,2025-03-28 21:34:23.679749,9
2,p80,912.351439,669.405648,676.198009,123456799,2025-03-28,2025-03-28 21:34:23.679749,10
3,p90,690.295279,490.883361,719.634796,123456799,2025-03-28,2025-03-28 21:34:23.679749,11
4,p92,617.563741,440.335754,766.835395,123456799,2025-03-28,2025-03-28 21:34:23.679749,12
5,p95,464.753707,363.030841,891.675155,123456799,2025-03-28,2025-03-28 21:34:23.679749,13
6,p99,741.091936,827.961471,1764.147173,123456799,2025-03-28,2025-03-28 21:34:23.679749,14
7,p1,3156.702064,2742.225599,2791.761490,123456799,2025-03-28,2025-03-28 21:34:23.679749,1
8,p5,2264.530481,1911.612032,1823.584030,123456799,2025-03-28,2025-03-28 21:34:23.679749,2
9,p10,1965.946367,1635.869394,1511.423338,123456799,2025-03-28,2025-03-28 21:34:23.679749,3


In [29]:
retrieve_data_from_sql(cwb_validation_forecasts_table_name).tail(14)


Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


,p1,p5,p10,p20,p30,p40,p50,p60,p70,p80,p90,p92,p95,p99,actual,date,applicant_id,date_added,transaction_time,sn
16,11099.436523,12097.791016,12396.582031,12674.372070,12828.990234,12943.431641,13058.488281,13181.825195,13314.469727,13469.220703,13706.059570,13774.414062,13979.580078,14778.019531,13407.184192,2024-08-17,123456799,2025-03-28,2025-03-28 21:34:23.68576,20
17,11469.277344,12010.881836,12301.628906,12592.440430,12742.440430,12843.066406,12974.786133,13112.508789,13257.260742,13442.697266,13685.190430,13762.562500,13906.329102,15258.925781,13378.803278,2024-08-18,123456799,2025-03-28,2025-03-28 21:34:23.68576,21
18,10691.055664,11805.420898,12194.450195,12543.597656,12722.395508,12887.959961,13030.468750,13185.990234,13341.001953,13499.634766,13788.231445,13880.478516,14089.421875,14892.662109,13367.922521,2024-08-19,123456799,2025-03-28,2025-03-28 21:34:23.68576,22
19,9967.979492,11950.076172,12248.497070,12597.790039,12840.882812,12999.372070,13166.669922,13319.660156,13465.890625,13673.850586,13991.957031,14082.481445,14275.222656,15355.040039,13362.229295,2024-08-20,123456799,2025-03-28,2025-03-28 21:34:23.68576,23
20,10940.541992,11944.611328,12295.567383,12665.735352,12931.201172,13105.017578,13282.841797,13454.547852,13649.924805,13909.327148,14237.101562,14410.593750,14782.831055,15873.652344,13353.429012,2024-08-21,123456799,2025-03-28,2025-03-28 21:34:23.68576,24
21,11996.731445,12619.526367,12896.437500,13141.045898,13279.532227,13397.390625,13504.800781,13621.554688,13719.712891,13846.266602,14070.141602,14220.875977,14390.271484,15161.860352,14222.218313,2024-08-06,123456799,2025-03-28,2025-03-28 21:34:23.68576,9
22,11974.449219,12614.335938,12867.881836,13116.179688,13249.853516,13363.357422,13460.426758,13568.437500,13682.957031,13833.604492,14057.160156,14140.574219,14318.597656,15115.815430,14042.506255,2024-08-07,123456799,2025-03-28,2025-03-28 21:34:23.68576,10
23,11927.807617,12539.968750,12796.181641,13044.366211,13197.587891,13307.511719,13412.373047,13512.394531,13621.483398,13768.584961,13984.731445,14049.753906,14169.312500,14806.905273,14040.872151,2024-08-08,123456799,2025-03-28,2025-03-28 21:34:23.68576,11
24,10096.961914,11829.586914,12391.847656,12822.984375,13020.672852,13247.330078,13387.436523,13556.572266,13726.405273,13961.082031,14381.054688,14520.123047,14763.408203,16035.200195,13320.715951,2024-08-22,123456799,2025-03-28,2025-03-28 21:34:23.68576,25
25,10847.524414,11986.284180,12338.743164,12811.528320,13059.422852,13271.954102,13440.019531,13648.614258,13877.545898,14110.059570,14529.999023,14697.545898,14986.169922,16307.218750,13318.248344,2024-08-23,123456799,2025-03-28,2025-03-28 21:34:23.68576,26
